In [4]:
def bernstein_constant(t, i, n):
    """ return the bernstein_constant for a Bezier curve
    
    t: floating point between [0,1]
    i: int between [0,n]
    n: number of control points
    """
    import math
    term1 = math.factorial(n)/(math.factorial(i)*math.factorial(n-i))
    term2 = ((1-t)**(n-i))*(t**i)
    return term1*term2

def bezier_point(t, control_points):
    """ Return RGB values for a point on a bezier curve given at t given a list of control points
    
    t: floating point between [0,1]
    control_points: list of control points, which should be in three dimensions (R,G,B)
    """
    import numpy as np
    control_points = list(map(np.asarray, control_points))
    result = [
        bernstein_constant(t, i, len(control_points)-1)*pnt for i, pnt in enumerate(control_points)
    ]
    return [int(sum(pnt)) for pnt in zip(*result)]

In [5]:
import plotly.plotly as py
import plotly.graph_objs as go

In [6]:
red = go.Scatter3d(
    x=[255],
    y=[0],
    z=[0],
    mode='markers',
    marker=dict(
        color='rgb(255, 0, 0)',
        size=12,
        symbol='circle',
        line=dict(
            color='rgb(255, 0, 0)',
            width=1
                )
        )
    )

green = go.Scatter3d(
    x=[0],
    y=[255],
    z=[0],
    mode='markers',
    marker=dict(
        color='rgb(0, 255, 0)',
        size=12,
        symbol='circle',
        line=dict(
            color='rgb(0, 255, 0)',
            width=1
                )
        )
    )

blue = go.Scatter3d(
    x=[0],
    y=[0],
    z=[255],
    mode='markers',
    marker=dict(
        color='rgb(0, 0, 255)',
        size=12,
        symbol='circle',
        line=dict(
            color='rgb(0, 0, 255)',
            width=1
                )
        )
    )


fig = go.Figure(data=[red, green, blue])
py.iplot(fig, filename='simple-3d-scatter')

In [8]:
import numpy as np

l = np.linspace(0, 1, 100)

r = [255, 0, 0]
g = [0, 255, 0]
b = [0, 0, 255]
line = [bezier_point(i, [r, b, g]) for i in l]

In [9]:
scatter = [go.Scatter3d(
        x=l[0],
        y=l[1],
        z=l[2],    
    mode='markers',
    marker=dict(
        color='rgb({0}, {1}, {2})'.format(l[0], l[1], l[2]),
        size=12,
        symbol='circle',
        line=dict(
            color='rgb({0}, {1}, {2})'.format(l[0], l[1], l[2]),
            width=1
                )
        )
        )
        for l in line
          ]

In [11]:
fig = go.Figure(data=[red, green, blue] + scatter)
py.iplot(fig, filename='simple-3d-scatter')


In [ ]:
bernstein_constant(1, 0, 2)*np.asarray(b)+bernstein_constant(1, 1, 2)*np.asarray(b)+bernstein_constant(1, 2, 2)*np.asarray(b)